In [1]:
import json
import pickle
import numpy as np
import sklearn
from gensim.models import KeyedVectors

import os
os.chdir("..")

w2v_model = KeyedVectors.load_word2vec_format('./Data/GoogleNews-vectors-negative300.bin', binary=True)

In [2]:
!pip install gensim

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 23.2 MB 45 kB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-4.2.0-cp36-cp36m-linux_x86_64.whl size=25519800 sha256=ef880166effcbc8fc7242aaa77b69fdb3d22269cfad4a7cf0eadab88d9fd513e
  Stored in directory: /home/sandwich/.cache/pip/wheels/44/1e/2b/b0056a533d057c3ed56c84fbdd79cca690496f4cd7c03c157c
Successfully built gensim


In [22]:
!pip install --upgrade scikit-learn -i https://pypi.tuna.tsinghua.edu.cn/simple

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [11]:
skip_token = ['to', 'and', 'a', 'of']
replace_token = {
    # 'jinrikisha':'rickshaw',
    # 'dhole':'dog',
    # 'madagascar':'Madagascar',
    # 'axe':'ax',
}

def get_w2v(string):
    
    string = string.lower().replace("_"," ").replace("-"," ").replace(":"," ").replace(","," ").replace("/", " ")
    vector = []
    for token in string.split():
        if token in replace_token:
            token = replace_token[token]
        if token not in skip_token:
            try:
                vector.append(w2v_model[token])
            except:
                continue
    vector = np.vstack(vector)
    # vector = np.sum(vector,axis=0)
    vector = np.mean(vector,axis=0)
    return vector.tolist()

In [24]:
from scipy.spatial.distance import cosine

def pairwise_cosine_distance(x, y):
    distances = []
    for xi in x:
        for yj in y:
            distance = cosine(xi, yj)
            distances.append(distance)
    return distances

In [28]:
target_label = "type the laptop" # enter the new activities you are interested in

## find related activities in our collected activities
Label2Node = json.load(open("./Data/Label2Node_batch1+2+3.json", "r"))
collected_activities = np.array([k for k in Label2Node.keys()])
collected_embed = []
for act in collected_activities:
    act = act.split('//')[1]
    try:
        vec = get_w2v(act)
        collected_embed.append(vec)
    except:
        collected_embed.append(np.zeros(300).tolist())
# collected_embed = np.array([get_w2v(act) for act in collected_activities])
collected_embed = np.array(collected_embed)
print("collected_embed", collected_embed.shape)

target_embed = np.array([get_w2v(target_label)])
print("target_embed", target_embed.shape)

# cosine_dist = sklearn.metrics.pairwise.cosine_distances(target_embed, collected_embed)
cosine_dist = pairwise_cosine_distance(target_embed, collected_embed)
index = np.argsort(cosine_dist)
print("top-5 related collected activities", collected_activities[index[:5]])

collected_embed (3666, 300)
target_embed (1, 300)
top-5 related collected activities ['HICO_DET//type_on laptop' 'Home_Action_Genome//type on laptop'
 'HICO_DET//read laptop' 'charades//Putting a laptop somewhere'
 'Home_Action_Genome//watch laptop']


/home/sandwich/.local/lib/python3.6/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [36]:
## get mapped nodes
TOPK = 5
mapped_nodes = []
for idx in index[:TOPK]:
    mapped_nodes.extend(np.unique(np.array(Label2Node[collected_activities[idx]])).tolist())
mapped_nodes = np.array(mapped_nodes)

unique_nodes, unique_counts = np.unique(mapped_nodes, return_counts=True)

print("mapped nodes, confidence")
for n, c in zip(unique_nodes, unique_counts):
    print("%s, %.1f"%(n, c/float(TOPK)))

mapped nodes, confidence
assessment-34.1, 0.6
comprehend-87.2-1, 0.2
crane-40.3.2, 0.2
create-26.4, 0.2
create-26.4-1, 0.2
create-26.4-1-1, 0.2
drive-11.5, 0.2
engender-27.1, 0.2
instr_communication-37.4.1, 0.4
learn-14-1, 0.2
learn-14-2-1, 0.2
peer-30.3, 0.2
put_spatial-9.2, 0.2
register-54.1-1-1, 0.2
scribble-25.2, 0.2
scribble-25.2-1, 0.4
see-30.1-1, 0.2
send-11.1-1, 0.2
sight-30.2, 0.2
spatial_configuration-47.6-1, 0.2
transcribe-25.4, 0.6
transfer_mesg-37.1.1-1, 0.2
turn-26.6.1, 0.2
use-105.1, 0.2
work-73.2, 0.2
